In [1]:
import numpy as np

from projectq import MainEngine
from projectq.ops import *
from projectq.meta import Compute, Uncompute
from projectq.backends import CircuitDrawer

In [2]:
# length of the string
n = 2

# define f(x), b for balanced, c for constant
oracle = "b"

# if the f(x) is balanced, set b
if oracle == "b":
    b = 3    # Not so sure,why it has to be 3:)

# if f(x) constant, set c = 0 or 1 randomly
if oracle == "c":
    c = np.ranom.randint(2)

In [3]:
# create Registers
# n qubits for querying the oracle, one qubit for storing the answer
drawing_engine = CircuitDrawer()
eng = MainEngine(backend = drawing_engine)
qureg = eng.allocate_qureg(n+1)

# all qubits are initialized to |0>, flip the last register qubit to the |1> state
# X | qureg[2]
# Barrier | qureg

# Apply Hadamard gates to all qubits
with Compute(eng):
    X | qureg[2]
    Barrier | qureg
    All(H) | qureg
    
Barrier | qureg

# Check f(x)
if oracle == "c":  # function is constant, c=1 apply X gate, c=0 do nothing
    if c == 1: 
        X | qureg[n]
        
else: # function is balanced, applys CNOT to qubit 0 and 1 
    for i in range(n):
        if (b & (1 << i)): # b = 3 "021", bitweise move, from 1 -> 2 ->0
            CNOT | (qureg[i],qureg[n])
            
Barrier | qureg

# apply Hadamard gates to the first register after querying the function 
# for i in range(n):
#     H | qureg[i]  
Uncompute(eng)   # how to use CustomUncompute(eng)... not so sure:) 

for i in range(n):
    Measure | qureg[i]
    
eng.flush()


In [4]:
print(drawing_engine.get_latex())
with open('Deutsch-Josza algorithm.tex','w') as fd:
    fd.write(drawing_engine.get_latex())

\documentclass{standalone}
\usepackage[margin=1in]{geometry}
\usepackage[hang,small,bf]{caption}
\usepackage{tikz}
\usepackage{braket}
\usetikzlibrary{backgrounds,shadows.blur,fit,decorations.pathreplacing,shapes}

\begin{document}
\begin{tikzpicture}[scale=0.8, transform shape]

\tikzstyle{basicshadow}=[blur shadow={shadow blur steps=8, shadow xshift=0.7pt, shadow yshift=-0.7pt, shadow scale=1.02}]\tikzstyle{basic}=[draw,fill=white,basicshadow]
\tikzstyle{operator}=[basic,minimum size=1.5em]
\tikzstyle{phase}=[fill=black,shape=circle,minimum size=0.1cm,inner sep=0pt,outer sep=0pt,draw=black]
\tikzstyle{none}=[inner sep=0pt,outer sep=-.5pt,minimum height=0.5cm+1pt]
\tikzstyle{measure}=[operator,inner sep=0pt,minimum height=0.5cm, minimum width=0.75cm]
\tikzstyle{xstyle}=[circle,basic,minimum height=0.35cm,minimum width=0.35cm,inner sep=-1pt,very thin]
\tikzset{
shadowed/.style={preaction={transform canvas={shift={(0.5pt,-0.5pt)}}, draw=gray, opacity=0.4}},
}
\tikzstyle{swapstyle}=[inne